In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


Looking in indexes: https://download.pytorch.org/whl/cu121



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Installing dependencies for training and augmentation, preprocessing
!pip install torch numpy transformers datasets tiktoken wandb tqdm
!pip install torchmetrics
!pip install imbalanced-learn
!pip install nlpaug numpy nltk
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
!pip install -U wandb


[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Ali\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ali\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!git clone https://github.com/caglarmert/DI725.git

fatal: destination path 'DI725' already exists and is not an empty directory.


In [4]:
%cd DI725/assignment_1

c:\Users\Ali\Desktop\DI725_e2231355_asg1\DI725\assignment_1


C:\Users\Ali\AppData\Roaming\Python\Python310\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [5]:
# Creating GPT sentiment class to convert decoder only GPT2 model to sentiment analysis model
import torch.nn as nn
import torch
from model import GPT
class GPTSentiment(GPT):
    def __init__(self, config):
        super().__init__(config)
        # We will add 3 classes for positive,neural and negative
        self.sentiment_head = nn.Linear(config.n_embd, 3)

    def forward(self, idx, targets=None):
        device = idx.device
        b, t = idx.size()
        # Assertion for block size (exists in original repo)
        assert t <= self.config.block_size, f"Cannot forward sequence of length {t}, block size is only {self.config.block_size}"

        pos = torch.arange(0, t, dtype=torch.long, device=device)
        tok_emb = self.transformer.wte(idx)
        pos_emb = self.transformer.wpe(pos)
        x = self.transformer.drop(tok_emb + pos_emb)
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)

        x = x.mean(dim=1)  # Take the mean embedding of the tokens
        logits = self.sentiment_head(x)  # Geting logits for 3-class classification

        loss = None
        if targets is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, targets)

        return logits, loss


In [6]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer
import nlpaug.augmenter.word as naw
from nltk.stem import WordNetLemmatizer
from nltk.corpus import RegexpTokenizer, stopwords
import string
nltk.download('stopwords')
# Creating dataloader for our dataset
# Dataloader contains preprocess, augment, padding all these

class SentimentDataset(Dataset):
    def __init__(self, df, tokenizer, max_length, padding=None, augment=False,take_only_customer=False,if_preprocess=True):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.label_to_int = {"negative": 0, "neutral": 1, "positive": 2}
        if padding is None:
            self.padding = "max_length"
        else:
            self.padding = False
        self.augment = augment
        self.if_take_customer = take_only_customer
        self.if_preprocess = if_preprocess
        if augment:
            self.aug = naw.SynonymAug(aug_src='wordnet')

        if self.if_preprocess:
          self.df = self.preprocess(self.df)

    def __len__(self):
        return len(self.df)

    # Remove Punctuation
    def remove_punctuation(self,text):
        no_punct = "".join([c for c in text if c not in string.punctuation])
        return no_punct

    # Remove Stopwords
    def remove_stopwords(self,text):
        words = [w for w in text if w not in stopwords.words('english')]
        return words

    def word_lemmatizer(self,text):
        lemmatizer = WordNetLemmatizer()
        return [lemmatizer.lemmatize(x) for x in text]

    def remove_numbers(self,text):
        return [x for x in text if not x.isdigit()]


    def take_only_customer(self,text):
        lines = text.strip().split("\n")
        customer_lines = [line.replace("Customer: ", "") for line in lines if line.startswith("Customer:")]
        customer_conversation = "\n".join(customer_lines)
        return customer_conversation

    def preprocess(self,df):
        df["conversation"] = df["conversation"].apply(lambda x: self.remove_punctuation(x))
        # Tokenize the text
        tokenizer = RegexpTokenizer(r'\w+')
        df["conversation"] = df["conversation"].apply(lambda x: tokenizer.tokenize(x.lower()))
        df["conversation"] = df["conversation"].apply(lambda x: self.remove_stopwords(x))
        df["conversation"] = df["conversation"].apply(lambda x: self.word_lemmatizer(x))
        df["conversation"] = df["conversation"].apply(lambda x: self.remove_numbers(x))

        df["conversation"] = df["conversation"].apply(lambda x: " ".join(x))
        return df

    def __getitem__(self, idx):

        row = self.df.iloc[idx]
        #if self.if_preprocess:
        #   text = self.preprocess(row)
        text = row["conversation"]
        if self.if_take_customer:
           text = self.take_only_customer(text)
        if self.augment:
           text = self.aug.augment(text)[0]
        sentiment_label = self.label_to_int[row["customer_sentiment"]]
        encoding = self.tokenizer.encode_plus(
          text,
          add_special_tokens=True,
          max_length=self.max_length,
          return_token_type_ids=False,
          padding=self.padding,
          return_attention_mask=True,
          return_tensors='pt',
          truncation=True
        )
        if len(encoding['input_ids'].flatten()) == 0:
          return {
          'input_ids': torch.tensor([0]),
          'attention_mask': encoding['attention_mask'].flatten(),
          'labels': torch.tensor(sentiment_label, dtype=torch.long)
          }
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(sentiment_label, dtype=torch.long)
        }


c:\Users\Ali\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ali\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Training script for Scratch and Pretrained init
"""
This training script can be run both on a single gpu in debug mode,
and also in a larger training run with distributed data parallel (ddp).

To run on a single GPU, example:
$ python train.py --batch_size=32 --compile=False

To run with DDP on 4 gpus on 1 node, example:
$ torchrun --standalone --nproc_per_node=4 train.py

To run with DDP on 4 gpus across 2 nodes, example:
- Run on the first (master) node with example IP 123.456.123.456:
$ torchrun --nproc_per_node=8 --nnodes=2 --node_rank=0 --master_addr=123.456.123.456 --master_port=1234 train.py
- Run on the worker node:
$ torchrun --nproc_per_node=8 --nnodes=2 --node_rank=1 --master_addr=123.456.123.456 --master_port=1234 train.py
(If your cluster does not have Infiniband interconnect prepend NCCL_IB_DISABLE=1)
"""

import os
import time
import math
import pickle
from contextlib import nullcontext
import matplotlib.pyplot as plt
from PIL import Image
import io

import numpy as np
import torch
import torch.nn as nn
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group

from model import GPTConfig, GPT
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import numpy as np
from torchmetrics import MeanMetric
from imblearn.over_sampling import RandomOverSampler
import math
import inspect
from dataclasses import dataclass
from transformers import GPT2ForSequenceClassification
import torch
import torch.nn as nn
from torch.nn import functional as F

nltk.download('averaged_perceptron_tagger_eng')

def configure_optimizers(model, weight_decay, learning_rate, betas, device_type):
        # start with all of the candidate parameters
        param_dict = {pn: p for pn, p in model.named_parameters()}
        # filter out those that do not require grad
        param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}
        # create optim groups. Any parameters that is 2D will be weight decayed, otherwise no.
        # i.e. all weight tensors in matmuls + embeddings decay, all biases and layernorms don't.
        decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
        nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
        optim_groups = [
            {'params': decay_params, 'weight_decay': weight_decay},
            {'params': nodecay_params, 'weight_decay': 0.0}
        ]
        num_decay_params = sum(p.numel() for p in decay_params)
        num_nodecay_params = sum(p.numel() for p in nodecay_params)
        print(f"num decayed parameter tensors: {len(decay_params)}, with {num_decay_params:,} parameters")
        print(f"num non-decayed parameter tensors: {len(nodecay_params)}, with {num_nodecay_params:,} parameters")
        # Create AdamW optimizer and use the fused version if it is available
        fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
        use_fused = fused_available and device_type == 'cuda'
        extra_args = dict(fused=True) if use_fused else dict()
        optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=betas, **extra_args)
        print(f"using fused AdamW: {use_fused}")

        return optimizer


# -----------------------------------------------------------------------------
# default config values designed to train a gpt2 (124M) on OpenWebText
# I/O
out_dir = 'out'
eval_interval = 20
eval_test_interval = 100
log_interval = 1
eval_iters = 200
if_SMOTE = True
eval_only = False # if True, script exits right after the first eval
always_save_checkpoint = True # if True, always save a checkpoint after each eval
#init_from = 'scratch' # 'scratch' or 'resume' or 'gpt2*'
init_from="scratch"
#init_from = 'scratch'
# wandb logging
wandb_log = True # disabled by default
wandb_project = 'DI725-HW1'
wandb_run_name = 'gpt2' + "_" + init_from # 'run' + str(time.time())
# data
#dataset = 'openwebtext'
gradient_accumulation_steps = 5 * 8 # used to simulate larger batch sizes
batch_size = 1 # if gradient_accumulation_steps > 1, this is the micro-batch size
block_size = 1024
# model
n_layer = 12
n_head = 12
n_embd = 768
dropout = 0.0 # for pretraining 0 is good, for finetuning try 0.1+
bias = False # do we use bias inside LayerNorm and Linear layers?
# adamw optimizer
learning_rate = 6e-4 # max learning rate
max_iters = 500 # total number of training iterations
weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.95
grad_clip = 1.0 # clip gradients at this value, or disable if == 0.0
# learning rate decay settings
learning_rate=3e-5
decay_lr = False # whether to decay the learning rate
warmup_iters = 2000 # how many steps to warm up for
lr_decay_iters = 600000 # should be ~= max_iters per Chinchilla
min_lr = 6e-5 # minimum learning rate, should be ~= learning_rate/10 per Chinchilla
# DDP settings
backend = 'nccl' # 'nccl', 'gloo', etc.
# system
#device = 'cuda' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1' etc., or try 'mps' on macbooks
device = "cuda"
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32', 'bfloat16', or 'float16', the latter will auto implement a GradScaler
compile = False # use PyTorch 2.0 to compile the model to be faster
# -----------------------------------------------------------------------------
config_keys = [k for k,v in globals().items() if not k.startswith('_') and isinstance(v, (int, float, bool, str))]
#exec(open('configurator.py').read()) # overrides from command line or config file
config = {k: globals()[k] for k in config_keys} # will be useful for logging
# -----------------------------------------------------------------------------

# various inits, derived attributes, I/O setup
ddp = int(os.environ.get('RANK', -1)) != -1 # is this a ddp run?
if ddp:
    init_process_group(backend=backend)
    ddp_rank = int(os.environ['RANK'])
    ddp_local_rank = int(os.environ['LOCAL_RANK'])
    ddp_world_size = int(os.environ['WORLD_SIZE'])
    device = f'cuda:{ddp_local_rank}'
    torch.cuda.set_device(device)
    master_process = ddp_rank == 0 # this process will do logging, checkpointing etc.
    seed_offset = ddp_rank # each process gets a different seed
    # world_size number of processes will be training simultaneously, so we can scale
    # down the desired gradient accumulation iterations per process proportionally
    assert gradient_accumulation_steps % ddp_world_size == 0
    gradient_accumulation_steps //= ddp_world_size
else:
    # if not ddp, we are running on a single gpu, and one process
    master_process = True
    seed_offset = 0
    ddp_world_size = 1
tokens_per_iter = gradient_accumulation_steps * ddp_world_size * batch_size * block_size
print(f"tokens per iteration will be: {tokens_per_iter:,}")

if master_process:
    os.makedirs(out_dir, exist_ok=True)
torch.manual_seed(1337 + seed_offset)
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
# note: float16 data type will automatically use a GradScaler
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

# poor man's data loader
data_dir = os.path.join('data', "random")


# init these up here, can override if init_from='resume' (i.e. from a checkpoint)
iter_num = 0
best_val_loss = 1e9



# attempt to derive vocab_size from the dataset
meta_path = os.path.join(data_dir, 'meta.pkl')
meta_vocab_size = None
if os.path.exists(meta_path):
    with open(meta_path, 'rb') as f:
        meta = pickle.load(f)
    meta_vocab_size = meta['vocab_size']
    print(f"found vocab_size = {meta_vocab_size} (inside {meta_path})")

# model init
model_args = dict(n_layer=n_layer, n_head=n_head, n_embd=n_embd, block_size=block_size,
                  bias=bias, vocab_size=None, dropout=dropout) # start with model_args from command line
if init_from == 'scratch':
    # init a new model from scratch
    print("Initializing a new model from scratch")
    # determine the vocab size we'll use for from-scratch training
    if meta_vocab_size is None:
        print("defaulting to vocab_size of GPT-2 to 50304 (50257 rounded up for efficiency)")
    model_args['vocab_size'] = meta_vocab_size if meta_vocab_size is not None else 50304
    gptconf = GPTConfig(**model_args)
    model_gpt = GPT(gptconf)
    model = GPTSentiment(gptconf)
elif init_from == 'resume':
    print(f"Resuming training from {out_dir}")
    # resume training from a checkpoint.
    ckpt_path = os.path.join(out_dir, 'ckpt.pt')
    checkpoint = torch.load(ckpt_path, map_location=device)
    checkpoint_model_args = checkpoint['model_args']
    # force these config attributes to be equal otherwise we can't even resume training
    # the rest of the attributes (e.g. dropout) can stay as desired from command line
    for k in ['n_layer', 'n_head', 'n_embd', 'block_size', 'bias', 'vocab_size']:
        model_args[k] = checkpoint_model_args[k]
    # create the model
    gptconf = GPTConfig(**model_args)
    model_gpt = GPT(gptconf)
    state_dict = checkpoint['model']
    # fix the keys of the state dictionary :(
    # honestly no idea how checkpoints sometimes get this prefix, have to debug more
    unwanted_prefix = '_orig_mod.'
    for k,v in list(state_dict.items()):
        if k.startswith(unwanted_prefix):
            state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
    model_gpt.load_state_dict(state_dict)
    model = GPTSentiment(model_gpt)
    iter_num = checkpoint['iter_num']
    best_val_loss = checkpoint['best_val_loss']
elif init_from.startswith('gpt2'):
    print(f"Initializing from OpenAI GPT-2 weights: {init_from}")
    # initialize from OpenAI GPT-2 weights
    override_args = dict(dropout=dropout)
    model_gpt = GPT.from_pretrained(init_from, override_args)
    # read off the created config params, so we can store them into checkpoint correctly
    for k in ['n_layer', 'n_head', 'n_embd', 'block_size', 'bias', 'vocab_size']:
        model_args[k] = getattr(model_gpt.config, k)
    gptconf = GPTConfig(**model_args)
    model = GPTSentiment(gptconf)
    model.load_state_dict(model_gpt.state_dict(),strict=False)



# crop down the model block size if desired, using model surgery
#if block_size < model.config.block_size:
#    model.crop_block_size(block_size)
#    model_args['block_size'] = block_size # so that the checkpoint will have the right value
model.to(device)

# initialize a GradScaler. If enabled=False scaler is a no-op
scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))

# optimizer
optimizer = configure_optimizers(model,weight_decay, learning_rate, (beta1, beta2), device_type)
if init_from == 'resume':
    optimizer.load_state_dict(checkpoint['optimizer'])
checkpoint = None # free up memory

# compile the model
if compile:
    print("compiling the model... (takes a ~minute)")
    unoptimized_model = model
    model = torch.compile(model) # requires PyTorch 2.0

# wrap model into DDP container
if ddp:
    model = DDP(model, device_ids=[ddp_local_rank])

# helps estimate an arbitrarily accurate loss over either split using many batches


# learning rate decay scheduler (cosine with warmup)
def get_lr(it):
    # 1) linear warmup for warmup_iters steps
    if it < warmup_iters:
        return learning_rate * it / warmup_iters
    # 2) if it > lr_decay_iters, return min learning rate
    if it > lr_decay_iters:
        return min_lr
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (it - warmup_iters) / (lr_decay_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # coeff ranges 0..1
    return min_lr + coeff * (learning_rate - min_lr)

# logging
if wandb_log and master_process:
    import wandb
    wandb.init(project=wandb_project, name=wandb_run_name, config=config)

# training loop


# Assuming 'filepath' is the path to your CSV file
train_df = pd.read_csv('./data/customer_service/train.csv')
test_df = pd.read_csv('./data/customer_service/test.csv')
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

# Initialize tokenizers, datasets, and dataloaders
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
if if_SMOTE:
    sm = RandomOverSampler(random_state=42)
    X_train_smote, y_train_smote = sm.fit_resample(train_df[["conversation"]],train_df["customer_sentiment"])
    train_df_smoted = pd.concat([X_train_smote, y_train_smote.reset_index(drop=True)], axis=1)
train_dataset = SentimentDataset(train_df_smoted, tokenizer, max_length=512,padding=False,augment=True,take_only_customer=False,if_preprocess=True)
val_dataset = SentimentDataset(val_df, tokenizer, max_length=512,padding=False,take_only_customer=False,if_preprocess=True)
test_dataset = SentimentDataset(test_df, tokenizer, max_length=512,padding=False,take_only_customer=False,if_preprocess=True)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
t0 = time.time()
local_iter_num = 0 # number of iterations in the lifetime of this process
raw_model = model.module if ddp else model # unwrap DDP container if needed
running_mfu = -1.0


def plot_confusion_matrix(conf_matrix, x_labels, y_labels):
    fig, ax = plt.subplots(figsize=(8, 6))
    cax = ax.matshow(conf_matrix, cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    fig.colorbar(cax)
    ax.set_xticklabels([''] + x_labels)
    ax.set_yticklabels([''] + y_labels)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.xticks(rotation=90)

    # Adding text to each cell
    for (i, j), val in np.ndenumerate(conf_matrix):
        ax.text(j, i, f'{val}', ha='center', va='center', color='red')

    plt.close(fig)  # Prevents the figure from being displayed in the notebook

    # Convert to PIL Image
    buf = io.BytesIO()
    fig.savefig(buf, format='png')
    buf.seek(0)
    img = Image.open(buf)

    return img


def estimate_loss(train_loader_iter,val_loader_iter,num_train,num_val):
    out = {"train":{},"val":{}}
    model.eval()
    all_preds = []
    all_labels = []
    for split in ['train', 'val']:



        if split=="train":
          if train_loader.batch_size == 1:
              iter_num= num_train
          else:
              iter_num = min( num_train // train_loader.batch_size + 1  , eval_iters // train_loader.batch_size + 1)
        else:
          if val_loader.batch_size == 1:
              iter_num = num_val
          else:
              iter_num = min( num_val // val_loader.batch_size + 1  , eval_iters // val_loader.batch_size + 1)

        losses = torch.zeros(iter_num)
        for k in range(iter_num):
            if split == "train":
                 try:
                    batch = next(train_loader_iter)
                    X = batch['input_ids'].to(device)
                    attention_mask = batch['attention_mask'].to(device)
                    Y = batch['labels'].to(device)
                 except:
                  continue
            else:
                 try:
                    batch = next(val_loader_iter)
                    X = batch['input_ids'].to(device)
                    attention_mask = batch['attention_mask'].to(device)
                    Y = batch['labels'].to(device)
                 except:
                  continue
            with ctx:
                logits, loss = model(X,Y)
                preds = torch.argmax(logits, dim=-1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(Y.cpu().numpy())
            losses[k] = loss.item()
        accuracy = accuracy_score(all_labels, all_preds)
        precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
        conf_matrix = confusion_matrix(all_labels, all_preds)
        out[split]["loss"] = losses.mean()
        out[split]["accuracy"] = accuracy
        out[split]["precision"] = precision
        out[split]["recall"] = recall
        out[split]["f1"] = f1
        out[split]["conf_matrix"] = conf_matrix

    model.train()
    return out


def estimate_loss_test(test_loader,num_samples):
    out = {"test":{}}
    test_loader_iter = iter(test_loader)
    model.eval()
    all_preds = []
    all_labels = []
    for split in ['test']:
        if test_loader.batch_size == 1:
          iter_num = num_samples
        else:
          iter_num = min( num_samples // test_loader.batch_size + 1  , eval_iters // test_loader.batch_size + 1)
        losses = torch.zeros(iter_num)
        print("Number of Samples Test = ",iter_num)
        for k in range(iter_num):
            if split == "test":
                 try:
                  batch = next(test_loader_iter)
                  X = batch['input_ids'].to(device)
                  attention_mask = batch['attention_mask'].to(device)
                  Y = batch['labels'].to(device)
                 except:
                  continue
            with ctx:
                logits,loss = model(X,Y)
                preds = torch.argmax(logits, dim=-1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(Y.cpu().numpy())
            losses[k] = loss.item()

        accuracy = accuracy_score(all_labels, all_preds)
        #print("All Labels",all_labels)
        precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
        conf_matrix = confusion_matrix(all_labels, all_preds)
        out[split]["loss"] = losses.mean()
        out[split]["accuracy"] = accuracy
        out[split]["precision"] = precision
        out[split]["recall"] = recall
        out[split]["f1"] = f1
        out[split]["conf_matrix"] = conf_matrix

    model.train()
    return out,all_labels,all_preds

train_loader_iter = iter(train_loader)
test_loader_iter = iter(test_loader)
val_loader_iter = iter(val_loader)
batch = next(train_loader_iter)
X = batch['input_ids'].to(device)
attention_mask = batch['attention_mask'].to(device)
Y = batch['labels'].to(device)

train_loss_metric = MeanMetric()
val_loss_metric = MeanMetric()
while True:

    # determine and set the learning rate for this iteration
    lr = get_lr(iter_num) if decay_lr else learning_rate
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    # evaluate the loss on train/val sets and write checkpoints
    if iter_num % eval_interval == 0 and master_process:
        out_res = estimate_loss(train_loader_iter,val_loader_iter,len(train_dataset), len(val_dataset))
        print(f"step {iter_num}: train loss {train_loss_metric.compute():.4f}, val loss {val_loss_metric.compute():.4f}")
        print(f"step val accuracy {out_res['val']['accuracy']:.4f}")

        if wandb_log:
            wandb.log({
                "iter": iter_num,
                "train/loss": train_loss_metric.compute(),
                "val/loss": val_loss_metric.compute(),
                "lr": lr,
                "val/acc":out_res['val']['accuracy'] ,
                #"mfu": running_mfu*100, # convert to percentage
            })

        if out_res['val']["loss"] < best_val_loss or always_save_checkpoint:
            best_val_loss = out_res['val']["loss"]
            if iter_num > 0:
                checkpoint = {
                    'model': raw_model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'model_args': model_args,
                    'iter_num': iter_num,
                    'best_val_loss': best_val_loss,
                    'config': config,
                }
                print(f"saving checkpoint to {out_dir}")
                torch.save(checkpoint, os.path.join(out_dir, 'ckpt.pt'))
    if iter_num % eval_test_interval == 0:
        out_res,all_labels,all_preds = estimate_loss_test(test_loader,len(test_dataset))
        test_acc = out_res['test']['accuracy']
        test_precision = out_res['test']['precision']
        test_recall = out_res['test']['recall']
        test_f1 = out_res['test']['f1']
        print(f"Test accuracy {test_acc:.4f}")
        print(f"Test precision {test_precision:.4f}")
        print(f"Test recall {test_recall:.4f}")
        print(f"Test F1 {test_f1:.4f}")
        print("Confusion Matrix")
        print("------------------")
        print(out_res['test']['conf_matrix'])

        if wandb_log:
            wandb.log({"Test_Accuracy": test_acc, "Test_Precision": test_precision, "Test_Recall": test_recall, "Test_F1_Score": test_f1})
            x_labels = ["negative", "neutral", "positive"]
            y_labels = ["negative", "neutral", "positive"]

            #wandb.log({"Confusion Matrix": wandb.plots.HeatMap(
            #x_labels, y_labels, out_res['test']['conf_matrix'], show_text=True
            #)})
            #conf_matrix_img = plot_confusion_matrix(out_res['test']['conf_matrix'], x_labels, y_labels)
            #print("All labels and all preds", all_labels, all_preds)
            wandb.log({"Confusion Matrix":wandb.plot.confusion_matrix(probs=None,
                                            y_true=all_labels, preds=all_preds,
                                            class_names=["negative", "neutral", "positive"])})

    if iter_num == 0 and eval_only:
        break

    # forward backward update, with optional gradient accumulation to simulate larger batch size
    # and using the GradScaler if data type is float16
    for micro_step in range(gradient_accumulation_steps):
        if ddp:
            # in DDP training we only need to sync gradients at the last micro step.
            # the official way to do this is with model.no_sync() context manager, but
            # I really dislike that this bloats the code and forces us to repeat code
            # looking at the source of that context manager, it just toggles this variable
            model.require_backward_grad_sync = (micro_step == gradient_accumulation_steps - 1)
        with ctx:
            logits, loss = model(X,Y)
            #print(logits)
            #print(Y)
            #print(loss)
            train_loss_metric.update(loss.item())
            loss = loss / gradient_accumulation_steps # scale the loss to account for gradient accumulation
        # immediately async prefetch next batch while model is doing the forward pass on the GPU
        try:
          batch = next(train_loader_iter)
        except:
          train_loader_iter = iter(train_loader)
          batch = next(train_loader_iter)
        X = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        Y = batch['labels'].to(device)
        # backward pass, with gradient scaling if training in fp16
        scaler.scale(loss).backward()
    # clip the gradient
    if grad_clip != 0.0:
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
    # step the optimizer and scaler if training in fp16
    scaler.step(optimizer)
    scaler.update()
    # flush the gradients as soon as we can, no need for this memory anymore
    optimizer.zero_grad(set_to_none=True)
    model.eval()
    with torch.no_grad():
        try:
          batch_val = next(val_loader_iter)
        except:
          val_loader_iter= iter(val_loader)
          batch_val = next(val_loader_iter)

        X_val = batch_val['input_ids'].to(device)
        attention_mask = batch_val['attention_mask'].to(device)
        Y_val = batch_val['labels'].to(device)
        logits_val , loss_val = model(X_val,Y_val)
        val_loss_metric.update(loss_val.item())
    model.train()




    # timing and logging
    t1 = time.time()
    dt = t1 - t0
    t0 = t1
    if iter_num % log_interval == 0 and master_process:
        # get loss as float. note: this is a CPU-GPU sync point
        # scale up to undo the division above, approximating the true total loss (exact would have been a sum)
        lossf = loss.item() * gradient_accumulation_steps
        if local_iter_num >= 5: # let the training loop settle a bit
            #mfu = raw_model.estimate_mfu(batch_size * gradient_accumulation_steps, dt)
            mfu = 0
            running_mfu = mfu if running_mfu == -1.0 else 0.9*running_mfu + 0.1*mfu
        print(f"iter {iter_num}: loss {lossf:.4f}, time {dt*1000:.2f}ms")
    iter_num += 1
    local_iter_num += 1

    # termination conditions
    if iter_num > max_iters:
        break

if ddp:
    destroy_process_group()


[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


tokens per iteration will be: 40,960
Initializing a new model from scratch
defaulting to vocab_size of GPT-2 to 50304 (50257 rounded up for efficiency)
number of parameters: 123.59M
number of parameters: 123.59M
num decayed parameter tensors: 51, with 124,356,864 parameters
num non-decayed parameter tensors: 26, with 19,203 parameters
using fused AdamW: True


<ipython-input-6-763ebe6df08c>:243: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: aliyigitbasaran (aliyigitbasaran-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: The ``compute`` method of metric MeanMetric was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)


step 0: train loss nan, val loss nan
step val accuracy 0.2943
Number of Samples Test =  30


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Test accuracy 0.3333
Test precision 0.1149
Test recall 0.3333
Test F1 0.1709
Confusion Matrix
------------------
[[ 0  0 10]
 [ 1  0  9]
 [ 0  0 10]]
iter 0: loss 1.3125, time 69357.87ms
iter 1: loss 11.3125, time 2794.59ms
iter 2: loss 0.2080, time 2814.23ms
iter 3: loss 2.8594, time 3199.42ms
iter 4: loss 0.7461, time 3235.71ms
iter 5: loss 0.7891, time 3001.80ms
iter 6: loss 1.7734, time 2991.91ms
iter 7: loss 1.3359, time 2973.93ms
iter 8: loss 1.5078, time 3199.47ms
iter 9: loss 1.1563, time 3029.41ms
iter 10: loss 1.7031, time 2839.33ms
iter 11: loss 1.7656, time 2972.85ms
iter 12: loss 1.3281, time 3122.76ms
iter 13: loss 1.3984, time 2880.09ms
iter 14: loss 0.4785, time 2829.61ms
iter 15: loss 1.8594, time 2891.76ms
iter 16: loss 1.4766, time 3030.59ms
iter 17: loss 0.7773, time 3346.64ms
iter 18: loss 1.9766, time 3533.66ms
iter 19: loss 1.5156, time 3111.10ms
step 20: train loss 1.5760, val loss 1.2840
step val accuracy 0.5089
saving checkpoint to out
iter 20: loss 1.3281, ti

In [7]:
# Hyperparameter Tuning Script
"""
This training script can be run both on a single gpu in debug mode,
and also in a larger training run with distributed data parallel (ddp).

To run on a single GPU, example:
$ python train.py --batch_size=32 --compile=False

To run with DDP on 4 gpus on 1 node, example:
$ torchrun --standalone --nproc_per_node=4 train.py

To run with DDP on 4 gpus across 2 nodes, example:
- Run on the first (master) node with example IP 123.456.123.456:
$ torchrun --nproc_per_node=8 --nnodes=2 --node_rank=0 --master_addr=123.456.123.456 --master_port=1234 train.py
- Run on the worker node:
$ torchrun --nproc_per_node=8 --nnodes=2 --node_rank=1 --master_addr=123.456.123.456 --master_port=1234 train.py
(If your cluster does not have Infiniband interconnect prepend NCCL_IB_DISABLE=1)
"""

import os
import time
import math
import pickle
from contextlib import nullcontext
import matplotlib.pyplot as plt
from PIL import Image
import io

import numpy as np
import torch
import torch.nn as nn
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group

from model import GPTConfig, GPT
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import numpy as np
from torchmetrics import MeanMetric
from imblearn.over_sampling import RandomOverSampler
import math
import inspect
from dataclasses import dataclass
from transformers import GPT2ForSequenceClassification
import torch
import torch.nn as nn
from torch.nn import functional as F
import wandb


def configure_optimizers(model, weight_decay, learning_rate, betas, device_type):
        # start with all of the candidate parameters
        param_dict = {pn: p for pn, p in model.named_parameters()}
        # filter out those that do not require grad
        param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}
        # create optim groups. Any parameters that is 2D will be weight decayed, otherwise no.
        # i.e. all weight tensors in matmuls + embeddings decay, all biases and layernorms don't.
        decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
        nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
        optim_groups = [
            {'params': decay_params, 'weight_decay': weight_decay},
            {'params': nodecay_params, 'weight_decay': 0.0}
        ]
        num_decay_params = sum(p.numel() for p in decay_params)
        num_nodecay_params = sum(p.numel() for p in nodecay_params)
        print(f"num decayed parameter tensors: {len(decay_params)}, with {num_decay_params:,} parameters")
        print(f"num non-decayed parameter tensors: {len(nodecay_params)}, with {num_nodecay_params:,} parameters")
        # Create AdamW optimizer and use the fused version if it is available
        fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
        use_fused = fused_available and device_type == 'cuda'
        extra_args = dict(fused=True) if use_fused else dict()
        optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=betas, **extra_args)
        print(f"using fused AdamW: {use_fused}")

        return optimizer


# -----------------------------------------------------------------------------
# default config values designed to train a gpt2 (124M) on OpenWebText
# I/O

def train():
  with wandb.init() as run:
    wandb_config = wandb.config
    out_dir = 'out'
    eval_interval = 20
    eval_test_interval = 100
    log_interval = 1
    eval_iters = 200
    if_SMOTE = wandb_config["if_smote"]
    eval_only = False # if True, script exits right after the first eval
    always_save_checkpoint = True # if True, always save a checkpoint after each eval
    #init_from = 'scratch' # 'scratch' or 'resume' or 'gpt2*'
    init_from="gpt2"
    #init_from = 'scratch'
    # wandb logging
    wandb_log = True # disabled by default
    # data
    #dataset = 'openwebtext'
    gradient_accumulation_steps = 5 * 8 # used to simulate larger batch sizes
    batch_size = 1 # if gradient_accumulation_steps > 1, this is the micro-batch size
    block_size = 1024
    # model
    n_layer = 12
    n_head = 12
    n_embd = 768
    dropout = 0.0 # for pretraining 0 is good, for finetuning try 0.1+
    bias = False # do we use bias inside LayerNorm and Linear layers?
    # adamw optimizer
    #learning_rate = 6e-4 # max learning rate
    max_iters = 500 # total number of training iterations
    weight_decay = 1e-1
    beta1 = 0.9
    beta2 = 0.95
    grad_clip = 1.0 # clip gradients at this value, or disable if == 0.0
    # learning rate decay settings
    learning_rate=wandb_config["learning_rate"]
    decay_lr = False # whether to decay the learning rate
    warmup_iters = 2000 # how many steps to warm up for
    lr_decay_iters = 600000 # should be ~= max_iters per Chinchilla
    min_lr = 6e-5 # minimum learning rate, should be ~= learning_rate/10 per Chinchilla
    # DDP settings
    backend = 'nccl' # 'nccl', 'gloo', etc.
    # system
    #device = 'cuda' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1' etc., or try 'mps' on macbooks
    device = "cuda"
    dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32', 'bfloat16', or 'float16', the latter will auto implement a GradScaler
    compile = False # use PyTorch 2.0 to compile the model to be faster
    # -----------------------------------------------------------------------------
    config_keys = [k for k,v in globals().items() if not k.startswith('_') and isinstance(v, (int, float, bool, str))]
    #exec(open('configurator.py').read()) # overrides from command line or config file
    config = {k: globals()[k] for k in config_keys} # will be useful for logging
    # -----------------------------------------------------------------------------

    # various inits, derived attributes, I/O setup
    ddp = int(os.environ.get('RANK', -1)) != -1 # is this a ddp run?
    if ddp:
        init_process_group(backend=backend)
        ddp_rank = int(os.environ['RANK'])
        ddp_local_rank = int(os.environ['LOCAL_RANK'])
        ddp_world_size = int(os.environ['WORLD_SIZE'])
        device = f'cuda:{ddp_local_rank}'
        torch.cuda.set_device(device)
        master_process = ddp_rank == 0 # this process will do logging, checkpointing etc.
        seed_offset = ddp_rank # each process gets a different seed
        # world_size number of processes will be training simultaneously, so we can scale
        # down the desired gradient accumulation iterations per process proportionally
        assert gradient_accumulation_steps % ddp_world_size == 0
        gradient_accumulation_steps //= ddp_world_size
    else:
        # if not ddp, we are running on a single gpu, and one process
        master_process = True
        seed_offset = 0
        ddp_world_size = 1
    tokens_per_iter = gradient_accumulation_steps * ddp_world_size * batch_size * block_size
    print(f"tokens per iteration will be: {tokens_per_iter:,}")

    if master_process:
        os.makedirs(out_dir, exist_ok=True)
    torch.manual_seed(1337 + seed_offset)
    torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
    torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
    device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
    # note: float16 data type will automatically use a GradScaler
    ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
    ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

    # poor man's data loader
    data_dir = os.path.join('data', "random")


    # init these up here, can override if init_from='resume' (i.e. from a checkpoint)
    iter_num = 0
    best_val_loss = 1e9



    # attempt to derive vocab_size from the dataset
    meta_path = os.path.join(data_dir, 'meta.pkl')
    meta_vocab_size = None
    if os.path.exists(meta_path):
        with open(meta_path, 'rb') as f:
            meta = pickle.load(f)
        meta_vocab_size = meta['vocab_size']
        print(f"found vocab_size = {meta_vocab_size} (inside {meta_path})")

    # model init
    model_args = dict(n_layer=n_layer, n_head=n_head, n_embd=n_embd, block_size=block_size,
                      bias=bias, vocab_size=None, dropout=dropout) # start with model_args from command line
    if init_from == 'scratch':
        # init a new model from scratch
        print("Initializing a new model from scratch")
        # determine the vocab size we'll use for from-scratch training
        if meta_vocab_size is None:
            print("defaulting to vocab_size of GPT-2 to 50304 (50257 rounded up for efficiency)")
        model_args['vocab_size'] = meta_vocab_size if meta_vocab_size is not None else 50304
        gptconf = GPTConfig(**model_args)
        model_gpt = GPT(gptconf)
        model = GPTSentiment(gptconf)
    elif init_from == 'resume':
        print(f"Resuming training from {out_dir}")
        # resume training from a checkpoint.
        ckpt_path = os.path.join(out_dir, 'ckpt.pt')
        checkpoint = torch.load(ckpt_path, map_location=device)
        checkpoint_model_args = checkpoint['model_args']
        # force these config attributes to be equal otherwise we can't even resume training
        # the rest of the attributes (e.g. dropout) can stay as desired from command line
        for k in ['n_layer', 'n_head', 'n_embd', 'block_size', 'bias', 'vocab_size']:
            model_args[k] = checkpoint_model_args[k]
        # create the model
        gptconf = GPTConfig(**model_args)
        model_gpt = GPT(gptconf)
        state_dict = checkpoint['model']
        # fix the keys of the state dictionary :(
        # honestly no idea how checkpoints sometimes get this prefix, have to debug more
        unwanted_prefix = '_orig_mod.'
        for k,v in list(state_dict.items()):
            if k.startswith(unwanted_prefix):
                state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
        model_gpt.load_state_dict(state_dict)
        model = GPTSentiment(model_gpt)
        iter_num = checkpoint['iter_num']
        best_val_loss = checkpoint['best_val_loss']
    elif init_from.startswith('gpt2'):
        print(f"Initializing from OpenAI GPT-2 weights: {init_from}")
        # initialize from OpenAI GPT-2 weights
        override_args = dict(dropout=dropout)
        model_gpt = GPT.from_pretrained(init_from, override_args)
        # read off the created config params, so we can store them into checkpoint correctly
        for k in ['n_layer', 'n_head', 'n_embd', 'block_size', 'bias', 'vocab_size']:
            model_args[k] = getattr(model_gpt.config, k)
        gptconf = GPTConfig(**model_args)
        model = GPTSentiment(gptconf)
        model.load_state_dict(model_gpt.state_dict(),strict=False)



    # crop down the model block size if desired, using model surgery
    #if block_size < model.config.block_size:
    #    model.crop_block_size(block_size)
    #    model_args['block_size'] = block_size # so that the checkpoint will have the right value
    model.to(device)

    # initialize a GradScaler. If enabled=False scaler is a no-op
    scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))

    # optimizer
    optimizer = configure_optimizers(model,weight_decay, learning_rate, (beta1, beta2), device_type)
    if init_from == 'resume':
        optimizer.load_state_dict(checkpoint['optimizer'])
    checkpoint = None # free up memory

    # compile the model
    if compile:
        print("compiling the model... (takes a ~minute)")
        unoptimized_model = model
        model = torch.compile(model) # requires PyTorch 2.0

    # wrap model into DDP container
    if ddp:
        model = DDP(model, device_ids=[ddp_local_rank])

    # helps estimate an arbitrarily accurate loss over either split using many batches


    # learning rate decay scheduler (cosine with warmup)
    def get_lr(it):
        # 1) linear warmup for warmup_iters steps
        if it < warmup_iters:
            return learning_rate * it / warmup_iters
        # 2) if it > lr_decay_iters, return min learning rate
        if it > lr_decay_iters:
            return min_lr
        # 3) in between, use cosine decay down to min learning rate
        decay_ratio = (it - warmup_iters) / (lr_decay_iters - warmup_iters)
        assert 0 <= decay_ratio <= 1
        coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # coeff ranges 0..1
        return min_lr + coeff * (learning_rate - min_lr)

    # logging
    #if wandb_log and master_process:
    #    import wandb
    #    wandb.init(project=wandb_project, name=wandb_run_name, config=config)

    # training loop


    # Assuming 'filepath' is the path to your CSV file
    train_df = pd.read_csv('./data/customer_service/train.csv')
    test_df = pd.read_csv('./data/customer_service/test.csv')
    train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

    # Initialize tokenizers, datasets, and dataloaders
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    tokenizer.pad_token = tokenizer.eos_token
    if if_SMOTE:
        sm = RandomOverSampler(random_state=42)
        X_train_smote, y_train_smote = sm.fit_resample(train_df[["conversation"]],train_df["customer_sentiment"])
        train_df_smoted = pd.concat([X_train_smote, y_train_smote.reset_index(drop=True)], axis=1)
        train_dataset = SentimentDataset(train_df_smoted, tokenizer, max_length=512,padding=False,augment=wandb_config["augment"],take_only_customer=False,if_preprocess=wandb_config["if_preprocess"])
    else:
        train_dataset = SentimentDataset(train_df, tokenizer, max_length=512,padding=False,augment=wandb_config["augment"],take_only_customer=False,if_preprocess=wandb_config["if_preprocess"])
    val_dataset = SentimentDataset(val_df, tokenizer, max_length=512,padding=False,take_only_customer=False,if_preprocess=wandb_config["if_preprocess"])
    test_dataset = SentimentDataset(test_df, tokenizer, max_length=512,padding=False,take_only_customer=False,if_preprocess=wandb_config["if_preprocess"])

    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    t0 = time.time()
    local_iter_num = 0 # number of iterations in the lifetime of this process
    raw_model = model.module if ddp else model # unwrap DDP container if needed
    running_mfu = -1.0


    def plot_confusion_matrix(conf_matrix, x_labels, y_labels):
        fig, ax = plt.subplots(figsize=(8, 6))
        cax = ax.matshow(conf_matrix, cmap=plt.cm.Blues)
        plt.title('Confusion Matrix')
        fig.colorbar(cax)
        ax.set_xticklabels([''] + x_labels)
        ax.set_yticklabels([''] + y_labels)
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.xticks(rotation=90)

        # Adding text to each cell
        for (i, j), val in np.ndenumerate(conf_matrix):
            ax.text(j, i, f'{val}', ha='center', va='center', color='red')

        plt.close(fig)  # Prevents the figure from being displayed in the notebook

        # Convert to PIL Image
        buf = io.BytesIO()
        fig.savefig(buf, format='png')
        buf.seek(0)
        img = Image.open(buf)

        return img


    def estimate_loss(train_loader_iter,val_loader_iter,num_train,num_val):
        out = {"train":{},"val":{}}
        model.eval()
        all_preds = []
        all_labels = []
        for split in ['train', 'val']:



            if split=="train":
              if train_loader.batch_size == 1:
                  iter_num= num_train
              else:
                  iter_num = min( num_train // train_loader.batch_size + 1  , eval_iters // train_loader.batch_size + 1)
            else:
              if val_loader.batch_size == 1:
                  iter_num = num_val
              else:
                  iter_num = min( num_val // val_loader.batch_size + 1  , eval_iters // val_loader.batch_size + 1)

            losses = torch.zeros(iter_num)
            for k in range(iter_num):
                if split == "train":
                    try:
                        batch = next(train_loader_iter)
                        X = batch['input_ids'].to(device)
                        attention_mask = batch['attention_mask'].to(device)
                        Y = batch['labels'].to(device)
                    except:
                      continue
                else:
                    try:
                        batch = next(val_loader_iter)
                        X = batch['input_ids'].to(device)
                        attention_mask = batch['attention_mask'].to(device)
                        Y = batch['labels'].to(device)
                    except:
                      continue
                with ctx:
                    logits, loss = model(X,Y)
                    preds = torch.argmax(logits, dim=-1)
                    all_preds.extend(preds.cpu().numpy())
                    all_labels.extend(Y.cpu().numpy())
                losses[k] = loss.item()
            accuracy = accuracy_score(all_labels, all_preds)
            precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
            conf_matrix = confusion_matrix(all_labels, all_preds)
            out[split]["loss"] = losses.mean()
            out[split]["accuracy"] = accuracy
            out[split]["precision"] = precision
            out[split]["recall"] = recall
            out[split]["f1"] = f1
            out[split]["conf_matrix"] = conf_matrix

        model.train()
        return out


    def estimate_loss_test(test_loader,num_samples):
        out = {"test":{}}
        test_loader_iter = iter(test_loader)
        model.eval()
        all_preds = []
        all_labels = []
        for split in ['test']:
            if test_loader.batch_size == 1:
              iter_num = num_samples
            else:
              iter_num = min( num_samples // test_loader.batch_size + 1  , eval_iters // test_loader.batch_size + 1)
            losses = torch.zeros(iter_num)
            print("Number of Samples Test = ",iter_num)
            for k in range(iter_num):
                if split == "test":
                    try:
                      batch = next(test_loader_iter)
                      X = batch['input_ids'].to(device)
                      attention_mask = batch['attention_mask'].to(device)
                      Y = batch['labels'].to(device)
                    except:
                      continue
                with ctx:
                    logits,loss = model(X,Y)
                    preds = torch.argmax(logits, dim=-1)
                    all_preds.extend(preds.cpu().numpy())
                    all_labels.extend(Y.cpu().numpy())
                losses[k] = loss.item()
            print("Labels and Preds")
            print(all_preds)
            print(all_labels)
            accuracy = accuracy_score(all_labels, all_preds)
            #print("All Labels",all_labels)
            precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
            conf_matrix = confusion_matrix(all_labels, all_preds)
            out[split]["loss"] = losses.mean()
            out[split]["accuracy"] = accuracy
            out[split]["precision"] = precision
            out[split]["recall"] = recall
            out[split]["f1"] = f1
            out[split]["conf_matrix"] = conf_matrix

        model.train()
        return out,all_labels,all_preds

    train_loader_iter = iter(train_loader)
    test_loader_iter = iter(test_loader)
    val_loader_iter = iter(val_loader)
    batch = next(train_loader_iter)
    X = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    Y = batch['labels'].to(device)

    train_loss_metric = MeanMetric()
    val_loss_metric = MeanMetric()
    while True:

        # determine and set the learning rate for this iteration
        lr = get_lr(iter_num) if decay_lr else learning_rate
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

        # evaluate the loss on train/val sets and write checkpoints
        if iter_num % eval_interval == 0 and master_process:
            out_res = estimate_loss(train_loader_iter,val_loader_iter,len(train_dataset), len(val_dataset))
            print(f"step {iter_num}: train loss {train_loss_metric.compute():.4f}, val loss {val_loss_metric.compute():.4f}")
            print(f"step val accuracy {out_res['val']['accuracy']:.4f}")

            if wandb_log:
                wandb.log({
                    "iter": iter_num,
                    "train/loss": train_loss_metric.compute(),
                    "val/loss": val_loss_metric.compute(),
                    "lr": lr,
                    "val/acc":out_res['val']['accuracy'] ,
                    #"mfu": running_mfu*100, # convert to percentage
                })

            if out_res['val']["loss"] < best_val_loss or always_save_checkpoint:
                best_val_loss = out_res['val']["loss"]
                if iter_num > 0:
                    checkpoint = {
                        'model': raw_model.state_dict(),
                        'optimizer': optimizer.state_dict(),
                        'model_args': model_args,
                        'iter_num': iter_num,
                        'best_val_loss': best_val_loss,
                        'config': config,
                    }
                    print(f"saving checkpoint to {out_dir}")
                    torch.save(checkpoint, os.path.join(out_dir, 'ckpt.pt'))
        if iter_num % eval_test_interval == 0:
            out_res,all_labels,all_preds = estimate_loss_test(test_loader,len(test_dataset))
            test_acc = out_res['test']['accuracy']
            test_precision = out_res['test']['precision']
            test_recall = out_res['test']['recall']
            test_f1 = out_res['test']['f1']
            print(f"Test accuracy {test_acc:.4f}")
            print(f"Test precision {test_precision:.4f}")
            print(f"Test recall {test_recall:.4f}")
            print(f"Test F1 {test_f1:.4f}")
            print("Confusion Matrix")
            print("------------------")
            print(out_res['test']['conf_matrix'])

            if wandb_log:
                wandb.log({"Test_Accuracy": test_acc, "Test_Precision": test_precision, "Test_Recall": test_recall, "Test_F1_Score": test_f1})
                x_labels = ["negative", "neutral", "positive"]
                y_labels = ["negative", "neutral", "positive"]

                #wandb.log({"Confusion Matrix": wandb.plots.HeatMap(
                #x_labels, y_labels, out_res['test']['conf_matrix'], show_text=True
                #)})
                #conf_matrix_img = plot_confusion_matrix(out_res['test']['conf_matrix'], x_labels, y_labels)
                #print("All labels and all preds", all_labels, all_preds)
                wandb.log({"Confusion Matrix":wandb.plot.confusion_matrix(probs=None,
                                                y_true=all_labels, preds=all_preds,
                                                class_names=["negative", "neutral", "positive"])})

        if iter_num == 0 and eval_only:
            break

        # forward backward update, with optional gradient accumulation to simulate larger batch size
        # and using the GradScaler if data type is float16
        for micro_step in range(gradient_accumulation_steps):
            if ddp:
                # in DDP training we only need to sync gradients at the last micro step.
                # the official way to do this is with model.no_sync() context manager, but
                # I really dislike that this bloats the code and forces us to repeat code
                # looking at the source of that context manager, it just toggles this variable
                model.require_backward_grad_sync = (micro_step == gradient_accumulation_steps - 1)
            with ctx:
                logits, loss = model(X,Y)
                #print(logits)
                #print(Y)
                #print(loss)
                train_loss_metric.update(loss.item())
                loss = loss / gradient_accumulation_steps # scale the loss to account for gradient accumulation
            # immediately async prefetch next batch while model is doing the forward pass on the GPU
            try:
              batch = next(train_loader_iter)
            except:
              train_loader_iter = iter(train_loader)
              batch = next(train_loader_iter)
            X = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            Y = batch['labels'].to(device)
            # backward pass, with gradient scaling if training in fp16
            scaler.scale(loss).backward()
        # clip the gradient
        if grad_clip != 0.0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
        # step the optimizer and scaler if training in fp16
        scaler.step(optimizer)
        scaler.update()
        # flush the gradients as soon as we can, no need for this memory anymore
        optimizer.zero_grad(set_to_none=True)
        model.eval()
        with torch.no_grad():
            try:
              batch_val = next(val_loader_iter)
            except:
              val_loader_iter= iter(val_loader)
              batch_val = next(val_loader_iter)

            X_val = batch_val['input_ids'].to(device)
            attention_mask = batch_val['attention_mask'].to(device)
            Y_val = batch_val['labels'].to(device)
            logits_val , loss_val = model(X_val,Y_val)
            val_loss_metric.update(loss_val.item())
        model.train()




        # timing and logging
        t1 = time.time()
        dt = t1 - t0
        t0 = t1
        if iter_num % log_interval == 0 and master_process:
            # get loss as float. note: this is a CPU-GPU sync point
            # scale up to undo the division above, approximating the true total loss (exact would have been a sum)
            lossf = loss.item() * gradient_accumulation_steps
            if local_iter_num >= 5: # let the training loop settle a bit
                #mfu = raw_model.estimate_mfu(batch_size * gradient_accumulation_steps, dt)
                mfu = 0
                running_mfu = mfu if running_mfu == -1.0 else 0.9*running_mfu + 0.1*mfu
            print(f"iter {iter_num}: loss {lossf:.4f}, time {dt*1000:.2f}ms")
        iter_num += 1
        local_iter_num += 1

        # termination conditions
        if iter_num > max_iters:
            break

    if ddp:
        destroy_process_group()

sweep_config = {
    'method': 'grid',  # Can be grid, random, or bayesian
    'metric': {
        'name': 'Test_Accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'if_smote': {
            'values': [True, False]
        },
        'if_preprocess': {
            'values': [True, False]
        },
        'learning_rate': {
            'values': [5e-5, 1e-4]
        },
        'augment': {
            'values': [True, False]
        }
    }
}

wandb_project = 'DI725-HW1-HyperParamTune'
wandb_run_name = 'gpt2' # 'run' + str(time.time())

sweep_id = wandb.sweep(sweep_config, project=wandb_project)
wandb.agent(sweep_id, train)


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: 9urflm9m
Sweep URL: https://wandb.ai/aliyigitbasaran-/DI725-HW1-HyperParamTune/sweeps/9urflm9m


wandb: Agent Starting Run: tzdtbbf4 with config:
wandb: 	augment: True
wandb: 	if_preprocess: True
wandb: 	if_smote: True
wandb: 	learning_rate: 5e-05
wandb: Currently logged in as: aliyigitbasaran (aliyigitbasaran-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


tokens per iteration will be: 40,960
Initializing from OpenAI GPT-2 weights: gpt2
loading weights from pretrained gpt: gpt2
forcing vocab_size=50257, block_size=1024, bias=True
overriding dropout rate to 0.0
number of parameters: 123.65M
number of parameters: 123.65M


C:\Users\Ali\AppData\Local\Temp\ipykernel_15488\2720948829.py:245: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))


...

step 40: train loss 0.8462, val loss 0.9722
step val accuracy 0.8888
saving checkpoint to out
iter 40: loss 0.2871, time 21631.10ms
iter 41: loss 0.0471, time 1714.39ms
iter 42: loss 0.1055, time 1702.03ms
iter 43: loss 0.0371, time 1701.18ms
iter 44: loss 0.1543, time 1810.28ms
iter 45: loss 0.0121, time 1849.90ms
iter 46: loss 0.3477, time 1686.87ms
iter 47: loss 0.0282, time 1720.38ms
iter 48: loss 0.1572, time 1736.47ms
iter 49: loss 0.0195, time 1699.92ms
iter 50: loss 0.5117, time 1813.01ms
iter 51: loss 0.1523, time 1830.92ms
iter 52: loss 0.1582, time 1668.92ms
iter 53: loss 0.0038, time 1720.72ms
iter 54: loss 0.0189, time 1713.49ms
iter 55: loss 0.0108, time 1704.76ms
iter 56: loss 0.0933, time 1729.98ms
iter 57: loss 0.2988, time 1730.47ms
iter 58: loss 1.2188, time 1714.33ms
iter 59: loss 0.0046, time 1733.26ms


c:\Users\Ali\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Ali\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


step 60: train loss 0.6509, val loss 0.8194
step val accuracy 0.9309
saving checkpoint to out
iter 60: loss 0.0171, time 21875.73ms
iter 61: loss 0.1729, time 1734.23ms
iter 62: loss 0.0017, time 1720.80ms
iter 63: loss 0.0093, time 1727.67ms
iter 64: loss 0.0135, time 1797.81ms
iter 65: loss 0.0019, time 1721.65ms
iter 66: loss 0.0095, time 1747.37ms
iter 67: loss 0.0007, time 1751.80ms
iter 68: loss 0.0012, time 1857.15ms
iter 69: loss 0.0105, time 1687.40ms
iter 70: loss 0.0198, time 1758.93ms
iter 71: loss 0.0000, time 1690.55ms
iter 72: loss 0.0334, time 1694.39ms
iter 73: loss 0.0009, time 1779.85ms
iter 74: loss 0.0012, time 1792.24ms
iter 75: loss 0.0004, time 1707.98ms
iter 76: loss 0.0305, time 1690.50ms
iter 77: loss 0.0454, time 1651.22ms
iter 78: loss 0.0046, time 1737.61ms
iter 79: loss 0.0006, time 1708.28ms
step 80: train loss 0.5176, val loss 0.7300
step val accuracy 0.9773
saving checkpoint to out
iter 80: loss 0.0386, time 21758.94ms
iter 81: loss 0.0214, time 1700.5

Test_Accuracy,▁█▇▇▇▇
Test_F1_Score,▁██▇▇▇
Test_Precision,▁█████
Test_Recall,▁█▇▇▇▇
iter,▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,█▆▅▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
val/acc,▁▆▇▇█▇████████████████████
val/loss,▅█▅▃▃▁▁▁▁▂▂▂▂▂▁▂▂▃▃▃▄▄▄▅▅
Test_Accuracy,0.7
Test_F1_Score,0.68323


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
